In [17]:
import warnings
warnings.simplefilter('ignore')

In [1]:
from urllib import request as req

data_dir = '/Users/ssv/newprolab/project01'
filename = 'gender_age_dataset.txt'
file_path = '/'.join([data_dir, filename])
url = 'http://data.newprolab.com/data-newprolab-com/project01/' + filename

# req.urlretrieve(url, file_path)

In [17]:
import pandas as pd
import json
from urllib.parse import urlparse
from datetime import datetime
from tqdm import tqdm_notebook
import pickle

In [96]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 20000)

In [4]:
df = pd.read_csv(file_path, sep='\t')

In [6]:
df['user_json'] = df['user_json'].apply(lambda x: json.loads(x))

In [7]:
df['url_counts'] = df['user_json'].apply(lambda x: len(x['visits']))

In [8]:
#  Формирование столбца, содержащего массив распарсенных урлов, 
#  тип ParseResult(scheme='http', netloc='zebra-zoya.ru', 
#  path='/200028-chehol-organayzer-dlja-macbook-11-grid-it.html', 
#  params='', query='utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun', fragment='')

df['url_list'] = df['user_json'].apply(lambda x: [urlparse(x['url']) for x in x['visits']])

In [9]:
#  Формирование столбца, содержащего массив дат, полученных из timestamp, 
#  тип datetime.datetime(2015, 3, 24, 10, 25, 44)
df['ts_list'] = df['user_json'].apply(lambda x: \
                                       [datetime.fromtimestamp(int(float(str(x['timestamp'])[:10] + \
                                                            '.' + str(x['timestamp'])[10:13]))) \
                                        for x in x['visits']])

In [10]:
# Выбор уникальных урлов из массива распарсенных урлов, ParseResult.netloc (str)
df['url_netloc'] = df['url_list'].apply(lambda x: list(set([y.scheme + '://' + y.netloc for y in x ])))

In [13]:
#========================================================

In [11]:
# Перекодировка урлов x.encode('idna').decode('idna') с очисткой от мусора 

i = 0
for items in tqdm_notebook(df['url_netloc']):    
    for netloc in items:
        try:
            netloc.encode('idna').decode('idna')
        except:           
            df['url_netloc'][i].remove(netloc)
            # print('netloc -- {} ==>'.format(i), netloc)
            netloc = ''
            continue     
    i += 1

A Jupyter Widget

In [46]:
import pickle
with open('url_for_scraping.pickle', 'wb') as file:
    pickle.dump(url_for_scraping, file)

In [ ]:
# with open('url_for_scraping.pickle', 'rb') as file:
#     url_for_scraping = pickle.load(file)

In [28]:
df_med_keywords = pd.DataFrame()
df_med_keywords[['uid', 'url_netloc']] = df[['uid', 'url_netloc']]
df_med_keywords['raw_keywords'] = ''

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

i = 0
for index, row in tqdm_notebook(df_med_keywords.iterrows()):
    keywords = []
    tag = ''
    for url in row['url_netloc']:
        try:
            html = BeautifulSoup(urlopen(url).read())        
            tag = html.find('meta', {'name':'description'})    
            if tag != None:
                keywords.append(tag.attrs['content'])
            tag = html.find('meta', {'name':'keywords'})    
            if tag != None:
                keywords.append(tag.attrs['content'])    
        except:
            continue
            
    df_med_keywords.loc[index]['raw_keywords'] = keywords

    i += 1
    if i == 1000 or index == 41137:
        i = 0
        filename = 'keywords_' + str(index) + '.pickle'
        with open(filename, 'wb') as file:
            pickle.dump(df_med_keywords, file)
            
with open('df_med_keywords.pickle', 'wb') as file:
    pickle.dump('df_med_keywords.pickle', file)            

In [102]:
41137 - 5000


36137

In [1]:
import pickle

In [100]:
with open('data/keywords_0000_0999.pickle', 'rb') as file:
    df_med_keywords_0000_0999 = pickle.load(file)

In [ ]:
with open('data/keywords_2000_6999.pickle', 'rb') as file:
    df_med_keywords_2000_6999 = pickle.load(file)

### från index 36138 till 41137 (5000)

In [73]:
with open('data/df_med_keywords_non_age.pickle', 'rb') as file:
    df_for_pred = pickle.load(file)

In [89]:
# delete rows
for index, row in df_for_pred['raw_keywords'].loc[36138:].iteritems():
    if len(row) == 0:
        df_for_pred.drop(index, inplace=True)      

In [90]:
df_for_pred.shape

(40621, 4)

In [101]:
df_med_keywords_999.loc[0000:3000]

,uid,url_netloc,raw_keywords
0,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"[http://news.yandex.ru, http://www.sotovik.ru,...",[Последние новости в Яндекс.Новостях: Актуальн...
1,d502331d-621e-4721-ada2-5d30b2c3801f,"[http://www.euroavtoprokat.ru, http://www.word...",[EuroAvtoprokat.Ru предлагает лучшие цены на п...
2,d50237ea-747e-48a2-ba46-d08e71dddfdb,"[http://cdn.etgdta.com, http://ru.oriflame.com...",[58994 кулинарных рецептов приготовления блюд ...
3,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"[http://translate-tattoo.ru, http://www.1obl.r...","[Сервис перевода татуировок надписей, живой пр..."
4,d503c3b2-a0c2-4f47-bb27-065058c73008,"[http://www.prommetizcomplect.ru, http://utro....","[{description}, {keywords}, Общественно-полити..."
5,d5090ddf-5648-487e-9caa-5cd8de4c60a4,"[https://cfire.mail.ru, http://petrovka-beauty...",[Cross Fire - захватывающий MMO-шутер со множе...
6,d50bcef8-16ff-4e80-a9d5-9343698f4e7c,"[https://www.avito.ru, http://yandex.ru, http:...","[Объявления на Avito — подать объявления., объ..."
7,d50e23dc-0cbd-4883-bd6c-23ce65b511cc,"[http://www.re-store.ru, http://www.gazprom.ru...",[Купить технику Apple и аксессуары в фирменном...
8,d50fdabb-4208-441b-ad35-bfa9c7a673ea,"[http://rusvesna.su, http://lifenews.ru, http:...",[Cобытия и новости 24 часа в сутки: эксклюзивн...
9,d511b480-23a6-4820-b439-9009ee63c94d,"[http://ololo.fm, http://www.tvcok.ru, http://...",[www.tvcok.ru - новый подход к онлайн фильмам!...
